In [274]:
import pandas as pd

In [275]:
df_w = pd.read_csv("all_ticks_wide.csv")
df_l = pd.read_csv("all_ticks_long.csv")

We created the function Is_Null() which takes a dataframe as input, then returns "True" if the dataframe has null values, "False" otherwise. 

In [276]:
def Is_Null(df):
    if df.isnull().astype(int).iloc[:,1].sum() > 0:
        return True
    else:
        return False

True

Then, we created another function Find_Outliers() which takes a dataframe and returns the outliers if the dataframe doesn't have any null values.

In [278]:
def Find_Outliers(df):
    if Is_Null(df):
        return False
    else:
        mean = df.iloc[:,1].mean()
        std = df.iloc[:,1].std()
        upper_limit = mean + 3*std
        lower_limit = mean - 3*std
        smaller_outliers = df[df.iloc[:,1] < lower_limit]
        larger_outliers = df[df.iloc[:,1] > upper_limit]
        return smaller_outliers, larger_outliers

Latly, we created the function df_Monthly() which takes a dataframe and a key indicating the year and the month key = (year,month), and returns a new dataframe consisting of the values in the original dataframe in that certain month of the year.

In [429]:
def df_Monthly(df, key):
    df_Monthly_dictionary = {}
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df_yearly = df["timestamp"].dt.year
    for i in range(2012,2020):
        df_current_year = df.loc[df_yearly == i]
        df_current_year_monthly = df_current_year["timestamp"].dt.month
        for k in range(1,13):
            
            
            df_Monthly_dictionary[i,k] = df_current_year.loc[df_current_year_monthly == k]
    
    return df_Monthly_dictionary.get(key)

The first stock we choose was BANVT. We created a new dataframe called df_BANVT consisting of two columns that are "timestamp" and "BANVT". (Son satırı siz n/a değerleri var mı diye bakabilesiniz diye koydum. İkinci satırdaki key değerinde yılı ve ayı değiştirerek deneyebilirsiniz.)

In [433]:
df_BANVT = df_w.loc[:,["timestamp","BANVT"]]
df_BANVT_monthly = df_Monthly(df_BANVT,key = (2012,12))
df_BANVT_monthly[df_BANVT_monthly.isna().any(axis=1)]

,timestamp,BANVT
1356,2012-12-03 13:45:00+00:00,NaN
1379,2012-12-04 13:00:00+00:00,NaN
1406,2012-12-05 13:15:00+00:00,NaN
1667,2012-12-19 14:15:00+00:00,NaN
1739,2012-12-24 12:45:00+00:00,NaN
1837,2012-12-28 09:45:00+00:00,NaN
1880,2012-12-31 15:30:00+00:00,NaN
